In [1]:
import numpy as np
import pandas as pd
from random import randint,sample
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
## cloning the repositry from the github to make the file easily shareable
!git clone https://github.com/guru-narayana/NNFL-assignment

Cloning into 'NNFL-assignment'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [3]:
## loading the data
data_frame = pd.read_excel("/content/NNFL-assignment/data5.xlsx").sample(frac = 1)
data = data_frame.to_numpy()

## seperating input and output data
input_prenorm_data = data[:,0:data.shape[1]-1]
op = data[:,data.shape[1]-1]

## normalising the data
max_data    = np.amax(input_prenorm_data,axis=0) ## max  values in each column
input_data  = input_prenorm_data/max_data
output_data = np.zeros((len(op),3))

#Auto encoding
for i in range(len(op)):
  output_data[i,int(op[i]-1)] = 1

In [4]:
trim_point_1 = int(input_data.shape[0]*0.7)
trim_point_2 = int(input_data.shape[0]*0.9)


input_train_data      = input_data[0:trim_point_1,:]
input_validation_data = input_data[trim_point_1:trim_point_2,:]
input_test_data       = input_data[trim_point_2:input_data.shape[0],:]

output_train_data      = output_data[0:trim_point_1]
output_validation_data = output_data[trim_point_1:trim_point_2]
output_test_data       = output_data[trim_point_2:input_data.shape[0]]

In [37]:
def perceptron(W,X,B=0):
  P = (X@W.T) + B
  return 1/(1 + np.exp(-P))


def confiusion_matrix_func(outpt,Y):
  pred = np.argmax(outpt, axis=1)+1
  output_test_data = np.argmax(Y, axis=1)+1
  confusion_matrix = np.zeros((3,3))
  count = 0
  for x in range(len(pred)):
    if(pred[x] == output_test_data[x]):
      count+=1
    if(pred[x] == 1):
      if(output_test_data[x] == 1):
        confusion_matrix[0][0]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][0]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][0]+=1
    elif(pred[x] == 2):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][1]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][1]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][1]+=1
    elif(pred[x] == 3):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][2]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][2]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][2]+=1
  c1 = confusion_matrix[0][0]/np.sum(confusion_matrix[0][:])
  c2 = confusion_matrix[1][1]/np.sum(confusion_matrix[1][:])
  c3 = confusion_matrix[2][2]/np.sum(confusion_matrix[2][:])
  A  = count/len(pred)
  return [c1,c2,c3,A]

def elm_auto_encoder(lamda):
  W1 = np.random.rand(input_data.shape[1]-1,input_data.shape[1])
  W2 = np.random.rand(input_data.shape[1]-2,input_data.shape[1]-1)
  B1 = np.random.rand(input_data.shape[1]-1)
  B2 = np.random.rand(input_data.shape[1]-2)
  H1 = perceptron(W1,input_train_data,B1)
  H2 = perceptron(W2,H1,B2)
  H1_valid = perceptron(W1,input_validation_data,B1)
  H2_valid = perceptron(W2,H1_valid,B2)
  H1_test = perceptron(W1,input_test_data,B1)
  H2_test = perceptron(W2,H1_test,B2)
  A = 0
  weights = 0
  for l in lamda:
    H = np.insert(H2.copy(),0,np.ones(H2.shape[0]),1)
    W = np.linalg.pinv((H.T@H)+np.identity(H.shape[1])/l)@ H.T @ output_train_data
    H_valid = np.insert(H2_valid,0,np.ones(H2_valid.shape[0]),1)
    pred = perceptron(W.T,H_valid,0)
    _,_,_,a = confiusion_matrix_func(pred,output_validation_data)
    if(a>A):
      weights = W
  H_test = np.insert(H2_test,0,np.ones(H2_test.shape[0]),1)
  pred = perceptron(weights.T,H_test,0)
  c1,c2,c3,A = confiusion_matrix_func(pred,output_test_data)
  print("class_1  Accuracy  : "+ str( c1))
  print("class_2  Accuracy  : "+ str( c2))
  print("class_3  Accuracy  : "+ str( c3))
  print( "Overall Accuracy  : "+ str( A))
  print("\n")

In [39]:

lamda = np.linspace(1e5,1e20,10) 
elm_auto_encoder(lamda)

class_1  Accuracy  : 0.8
class_2  Accuracy  : 1.0
class_3  Accuracy  : 1.0
Overall Accuracy  : 0.9523809523809523


